<a href="https://colab.research.google.com/github/datavzch/GANWS21_DEMO/blob/main/Training_Stylegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Requirements and Preparation**

In [ ]:
%tensorflow_version 1.x

In [ ]:
%pip install Google-Colab-Transfer

In [ ]:
!apt-get install imagemagick

**Mount Google Drive**

In [ ]:
import colab_transfer

gd = colab_transfer.get_path_to_home_of_google_drive()
lm = colab_transfer.get_path_to_home_of_local_machine()

colab_transfer.mount_google_drive()

**Clone Stylegan2-ada  Repository**

In [ ]:
%cd /content/
%rm -rf stylegan2-ada/
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada/
!git checkout google-colab
%cd /content/

## **Create Personal Dataset**

In [ ]:
%mkdir basefordataset
%cd basefordataset
%mkdir category1
%cd category1

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
%cd ../
%ls

**Resize Images and generate tfrecords**

In [ ]:
!mogrify -resize !256x256 basefordataset/*/*.png 

In [ ]:
!python stylegan2-ada/dataset_tool.py create_from_images ./dataset ./basefordataset/category1

In [ ]:
!tar -czvf dataset.tar.gz ./dataset/

**Copy resulting dataset folder to Google drive**

In [ ]:
colab_transfer.copy_file(
    file_name="dataset.tar.gz",
    source="./",
    destination="./drive/My Drive/Datasets/",
)

**Copy  dataset from drive to stylegan folder**

In [ ]:
colab_transfer.copy_file(
    file_name="dataset.tar.gz",
    source="./drive/My Drive/Datasets/",
    destination="./stylegan2-ada/Datasets/",
)

In [ ]:
!tar -C ./stylegan2-ada/Datasets/ -xvf ./stylegan2-ada/Datasets/dataset.tar.gz

In [ ]:
%cd ..

# **Training Stylegan**

In [ ]:
!python ./stylegan2-ada/train.py \
  --outdir='../drive/My Drive/training-runs' \
  --snap=6 \
  --data='./dataset' \
  --mirror=true \
  --metrics=none \
  --cfg=auto_no_ramp \
  --cfg_map=8 \
  --kimg=5000 \
  --resume=lsundog256 \
  --freezed=10 \

**Resume from snapshot**

In [ ]:
!python stylegan2-ada/train.py \
 --outdir='/content/drive/My Drive/training-runs' \
 --snap=6 \
 --data='/content/datasets/steam' \
 --mirror=true \
 --metrics=none \
 --cfg=auto_no_ramp \
 --cfg_map=8 \
 --kimg=5000 \
 --resume=latest \
 --p=0.713 \
 --freezed=10 \